In [ ]:
# imports
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
# read advances data

advances_df = pd.read_parquet('data/advances.parquet', engine='fastparquet')
advances_df.head()

,request_id,underwritten_at,disbursed_at,amount,user_id
0,b'-\xf1\x07\x97\xc6)M\xda\x9c\xe4\x03{}N.M',2024-11-22 17:41:16.852959,2024-11-22 19:45:23.424420,25.0,b'\x94\xf7\xe5\xccL\xacG\xb0\x9emn\xdc\x86(\x1...
1,"b""\x8a\xa7T\x94\x98\xe1N\xcf\x98'\xe3\xe1\xeb\...",2025-02-12 14:25:34.060244,2025-02-12 15:48:40.796549,25.0,b'\x86x\xc3\x9aX\x04K\xe8\x93\x95\xcd\xfb! q%'
2,"b'\xf0i\x93\x03""\xeaB\x94\xbe\xda\xd7O\xf5\xaa...",2025-03-14 15:58:58.309632,2025-03-14 18:56:25.940474,25.0,b'\xb6:\x85\xa7TyI\x8f\x8e\xd2!d%\x12\x92]'
3,b'1\x9fy\x14\xba\x95E\x12\xa2\x00\x90\x86\xd5\...,2024-11-22 19:54:14.267373,2024-11-23 00:44:50.093139,25.0,b'f\xf3\xb42\xc9\xfeH\xd5\xa0\xb7p[\xa4\x05c\xf1'
4,b'Nt\xa9\xaa\xe5\xa0@N\x85\x90\xf6\xbe\x06x\x1...,2025-03-20 22:33:59.727517,2025-03-20 23:23:32.324881,25.0,b'\xf2\xa9\x96\xa5P\xf0G1\xa9ho{\xee?\x86W'


In [ ]:
# ensure underwritten_at is a datetime type
advances_df['underwritten_at'] = pd.to_datetime(advances_df['underwritten_at'])

# get first entry per user based on underwritten_at
first_entries = advances_df.sort_values('underwritten_at').drop_duplicates('user_id', keep='first')
# first_entries.to_csv('cleaned_advances.csv', index=False)

In [10]:
# Verify the results
print(f"Original dataframe: {len(advances_df)} rows")
print(f"First entries dataframe: {len(first_entries)} rows")
print(f"Unique users in original: {advances_df['user_id'].nunique()}")
print(f"Users in first entries: {len(first_entries)}")

Original dataframe: 57958 rows
First entries dataframe: 16691 rows
Unique users in original: 16691
Users in first entries: 16691


In [ ]:
# read labels data
table = pq.read_table('data/labels.parquet')
labels_df = table.to_pandas(ignore_metadata=True)
print(labels_df.head())

                                          advance_id  repaid_full_30d  \
0       b'y\xf1D\xeb\x04\xffO\\\x84Or"\xc6\x87t\xed'             True   
1  b'\x05\x07\x96\xbe2sE2\x80\x11\xde\x88\xea\x9d...             True   
2             b'sf1\xfdC\\AW\x86)W\xe6\xb9\xdb\xe8<'             True   
3        b'\x9b~:\xb5\xdb\xd6Nh\x9e\xb0[\xcb\xa7QId'             True   
4  b'\x9c!\x91/\xa91M\x12\xb7\xf7\xeb\xd3o\x00\xd...             True   

   repaid_pct_30d                                days_to_first_repay  
0             2.0   b'\x00\x00\x00\x00\x06\x00\x00\x00\xcfz\xb0\x01'  
1             2.0     b'\x00\x00\x00\x00\r\x00\x00\x00y\xd6\xd1\x02'  
2             2.0  b'\x00\x00\x00\x00\x00\x00\x00\x00\n\xdd\xfa\x04'  
3             2.0      b'\x00\x00\x00\x00\x01\x00\x00\x00:\x07c\x04'  
4             2.0  b'\x00\x00\x00\x00\x05\x00\x00\x00\x1f\xaf\x04...  


In [17]:
merged_df = first_entries.merge(
    labels_df,
    left_on='request_id',
    right_on='advance_id',
    how='inner'
)

# Check the result
print("Merged DataFrame Info:")
print(f"Shape: {merged_df.shape}")
print(f"\nColumns: {merged_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(merged_df.head())

Merged DataFrame Info:
Shape: (16315, 9)

Columns: ['request_id', 'underwritten_at', 'disbursed_at', 'amount', 'user_id', 'advance_id', 'repaid_full_30d', 'repaid_pct_30d', 'days_to_first_repay']

First few rows:
                                          request_id  \
0       b'u\xab>6\xa8\xc7E\xaf\x8c\x85\xd6\t9g\xe5&'   
1    b'\x95\xe6\xb4A\tdH\x8b\x88\xe7P:V\xe8\x9d\xf6'   
2     b'x\x97H\xf6w\x8eE\xab\xafK\xf3\xba\x94\xcb,y'   
3  b'2\x00/\x19\xa7\x7fK\x05\xa9\xb2>\x8a\xdcG\x0...   
4  b':\xb1\x02\x86*vB\xc9\xb4\xd2L\xe3\xe7\xabd\xd5'   

             underwritten_at               disbursed_at  amount  \
0 2024-03-09 18:41:20.327833 2024-03-10 02:01:20.488311    25.0   
1 2024-04-15 21:38:48.312520 2024-04-15 22:39:40.439586    25.0   
2 2024-04-19 07:22:19.237438 2024-04-19 16:20:55.435149    25.0   
3 2024-05-14 20:13:21.452987 2024-05-14 20:14:24.636421    25.0   
4 2024-05-16 22:29:08.763494 2024-05-16 22:31:07.360124    25.0   

                                             us

In [18]:
merged_df.to_csv('cleaned_advances_data.csv', index=False)